DATA TO PULL:
    - Projected and Actual points by week
        a) Projected only available for past and upcoming week (if in season)
        b) Need to pull this at the week/team/player level and aggregate in order to get projected points
        c) HAVE CODE FOR THIS
    - Indicator if week is part of regular season or playoffs
        a) HAVE CODE FOR THIS
    - Team IDs, Names, and Divisions (mapped to team)
    - Number of teams (can do this several different ways)
    
    
TODO:
    - Pull team data (team level)
        a) teamId, divisionId, full name, nickname
    - Pull settings data (league level)
        a) see settingsData cell
    - Pull matchup data (team/week level)
        a) teamId, scoringPeriod, matchupPeriod, score
        b) This will primarily be used to verify the team/week aggregate from the week/team/player data
            1. Note: Concern here is the possibility of non-player scores (e.g. "home team bonus") throwing
            things off
    - Update league finder function to only include leagues that actually have data
        a) Consider looking for more leagues (should only have around 500 or so that can actually be used)
    - Data checks
        a) Get freqs for the different settings variables
            1. This should help provide more clarity regarding what each variable actually is
        b) Compare week/team score of matchup data to agg of week/team/player data
    - Data processing
        a) Agg week/team/player data

In [89]:
# team_data = pull_data(2020, 48347143, params=(('view', 'mTeam')))
# team_settings = pull_data(2020, 48347143, params=(("view", "mSettings")))

teamObj = teamData(team_data)
settingsObj= settingsData(team_settings)

# teamObj.df_team_data
# teamObj.df_members_names

# print(teamObj.df_team_data)
settingsObj.playoffSeedingRule

'TOTAL_POINTS_SCORED'

In [57]:
#####################################################################################
################################# General Functions #################################
#####################################################################################

def get_league_ids(leagues_found_csv='Leagues_Found.csv'):
    """ Returns dataframe containing all of the league IDs found """

    df_leagues_found = pd.read_csv(leagues_found_csv)

    df_leagues_found = df_leagues_found.loc[df_leagues_found['seasonId'] == 2020]

    league_ids = list(df_leagues_found['leagueId'])
    
    return league_ids


def convert_tuple_to_list(tuple_var):
    """ 
    Converts tuple to a list
    Note: This isn't really necessary, but accounts for 1D tuple cases so i'm using it
    """
    
    list_var = []
    
    # Need to process 1D and 2D tuples differently
    if type(tuple_var[0]) is tuple:
        for value in tuple_var:

            dict_key = value[0]
            dict_value = value[1]

            list_var.append([dict_key, dict_value])
    else:
        dict_key = tuple_var[0]
        dict_value = tuple_var[1]

        list_var.append([dict_key, dict_value])
            
    return list_var


def convert_dict_to_list(dict_var):
    """ Converts dictionary to a 2D list """
    
    list_var = []
    for param, param_value in dict_var.items():
        list_var.append([param, param_value])
        
    return list_var


def rearrange_list_by_val(input_list, move_vals_list):
    """ 
    Moves the 'vals' in the 'list_var' to the front
    
    Note: This is primarily used to rearrange the columns in a dataframe. Due to this, the lists this
    will be used for will be relatively small, so I'm not worrying about efficiency
    """
    
    move_vals_list_len = len(move_vals_list)
  
    for i in range(1, move_vals_list_len + 1):
        if move_vals_list_len > 0:
            # Start with the back of the list in order to properly arrange the values
            list_index = -1 * i
        else:
            list_index = 0
            
        move_val = move_vals_list[list_index]
        
        for j, match_val in enumerate(input_list):
            if match_val == move_val:
                move_val = input_list.pop(j)
                input_list.insert(0, move_val)
            else:
                pass
                
    return input_list


def rearrange_df_columns(df, move_columns_list):
    """ Returns a dataframe with its columns rearranged according to the passed 'move_columns_list' """
    
    df = df.copy()
    
    df_cols = list(df.columns)
    
    df_cols_new = rearrange_list_by_val(df_cols, move_columns_list)
    
    df = df[df_cols_new]
    
    return df


# check = rearrange_df_columns(matchup_data, ['Team', 'Player', 'fff'])

# check
    

,Team,Player,Week,Slot,Pos,Status,Proj,Actual
0,1,Alvin Kamara,1,2,RB,ACTIVE,17.406407,21.20
1,1,Travis Kelce,1,6,TE,ACTIVE,14.886366,14.00
2,1,Chris Carson,1,2,RB,ACTIVE,14.635647,21.60
3,1,Calvin Ridley,1,4,WR,ACTIVE,11.783908,32.40
4,1,Keenan Allen,1,23,Flex,ACTIVE,11.671297,5.70
...,...,...,...,...,...,...,...,...
200,12,Austin Hooper,1,20,Bench,ACTIVE,6.595182,2.50
201,12,Nyheim Hines,1,20,Bench,ACTIVE,7.610436,23.30
202,12,Colts D/ST,1,16,Def,NA,8.107672,4.00
203,12,Derek Carr,1,20,Bench,ACTIVE,18.112839,13.56


In [53]:
import requests
import pandas as pd
import numpy as np

from ratelimit import limits, sleep_and_retry

pd.options.display.max_columns = None
pd.options.display.max_rows = 100

slotcodes = {
    0 : 'QB', 2 : 'RB', 4 : 'WR',
    6 : 'TE', 16: 'Def', 17: 'K',
    20: 'Bench', 21: 'IR', 23: 'Flex'
}


@sleep_and_retry
@limits(calls=6000, period=600)
def pull_data(season_id, league_id, params=None):
    """ Returns a JSON object containing the data pulled APIs url """

    if params == None:
        params = []
        
    if season_id < 2020:
        url = "https://fantasy.espn.com/apis/v3/games/ffl/leagueHistory/" + \
              str(league_id) + "?seasonId=" + str(season_id)
    else:
        url = "https://fantasy.espn.com/apis/v3/games/ffl/seasons/" + \
              str(season_id) + "/segments/0/leagues/" + str(league_id)
    
    # Passing the dict_params directly to the request_params of the requests.get method was
    # resulting in certain pulls retrieving unspecified data.
    # So, I'm directly applying those parameters to the URL string to prevent this
    # Note: This was likely happening due to duplicate keys being used (e.g. "view") in the dict
    
    if type(params) is tuple:
        params = convert_tuple_to_list(params)
        
    if type(params) is dict:
        params = convert_dict_to_list(params)
    
    for full_param in params:
        param = str(full_param[0])
        param_value = str(full_param[1])
        
        if url.find("?") == -1:
            url = url + "?" + param + "=" + param_value
        else:
            url = url + "&" + param + "=" + param_value
    
    r = requests.get(url)
    
    status_code = r.status_code
    
    if r.status_code == 200:
        pass
    else:
        if r.status_code == 429:
            print("429 error")

        return None   
    
    # 2020 url returns JSON object while prior seasonIds return it in a list 
    if season_id < 2020:
        d = r.json()[0]
    else:
        d = r.json()
        
    r.close()
        
    return d


def create_week_team_player_df(season_id, league_id, week_start=1, week_end=17):
    data = []
    for week in range(week_start, week_end + 1):

        request_params = (("view", "mMatchup"), ("view", "mMatchupScore"), ("scoringPeriodId", week))

        d = pull_data(season_id, league_id, params=request_params)

        try:            
            for tm in d['teams']:
                tmid = tm['id']
                for p in tm['roster']['entries']:
                    name = p['playerPoolEntry']['player']['fullName']

                    slot = p['lineupSlotId']
                    pos  = slotcodes[slot]

                    # injured status (need try/exc bc of D/ST)
                    inj = 'NA'
                    try:
                        inj = p['playerPoolEntry']['player']['injuryStatus']
                    except:
                        pass

                    # projected/actual points
                    proj, act = None, None
                    for stat in p['playerPoolEntry']['player']['stats']:
                        if stat['scoringPeriodId'] != week:
                            continue
                        if stat['statSourceId'] == 0:
                            act = stat['appliedTotal']
                        elif stat['statSourceId'] == 1:
                            proj = stat['appliedTotal']

                    data.append([week, tmid, name, slot, pos, inj, proj, act])
                    
        except:
            pass
                
    columns = ['Week', 'Team', 'Player', 'Slot', 'Pos', 'Status', 'Proj', 'Actual']
                
    data = pd.DataFrame(data, columns=columns)
                
    return data

# 28056918
matchup_data = create_week_team_player_df(2020, 48347143, week_end=1)
# settings_data = pull_data(2019, 28056918, params=(("view", "mSettings")))

matchup_data.loc[matchup_data['Team']==1]

,Week,Team,Player,Slot,Pos,Status,Proj,Actual
0,1,1,Alvin Kamara,2,RB,ACTIVE,17.406407,21.20
1,1,1,Travis Kelce,6,TE,ACTIVE,14.886366,14.00
2,1,1,Chris Carson,2,RB,ACTIVE,14.635647,21.60
3,1,1,Calvin Ridley,4,WR,ACTIVE,11.783908,32.40
4,1,1,Keenan Allen,23,Flex,ACTIVE,11.671297,5.70
5,1,1,Will Fuller V,4,WR,ACTIVE,11.766370,18.20
6,1,1,J.K. Dobbins,20,Bench,ACTIVE,7.680690,14.20
7,1,1,Henry Ruggs III,20,Bench,QUESTIONABLE,8.207652,8.10
8,1,1,CeeDee Lamb,20,Bench,ACTIVE,8.513200,8.40
9,1,1,Drew Brees,0,QB,ACTIVE,18.545038,14.40


In [4]:
"""
Additional data to pull:
    - playoffSeedingRule
    - playoffSeedigRuleBy
    - num_playoff_teams
    - firstScoringPeriod
    - finalScoringPeriod
    - scoring_type
    - reg_season_matchup_tiebreaker
    - playoff_matchup_tiebreaker
    - home_team_bonus
"""

class settingsData():
    """ 
    Pulls the relevant league settings 
    
    Note: example of what to use for 'raw_mTeam_data': 
    pull_data(2019, 28056918, params=(("view", "mSettings")))
    
    TODO: update this so that this is part of the class and all that needs passed is the league id and year
    """
    
    def __init__(self, season_id, league_id):
        
        raw_settings_data = pull_data(season_id, league_id, params=(("view", "mSettings")))
        
        self.season_id = season_id
        self.league_id = league_id
        
        schedule_settings = raw_settings_data['settings']['scheduleSettings']
        scoring_settings = raw_settings_data['settings']['scoringSettings']
        status_settings = raw_settings_data['status']
        
        self.league_name = raw_settings_data['settings']['name']
        
        self.num_divisions = len(schedule_settings['divisions'])
        self.df_divisions = self._pull_divisions(raw_settings_data)
        
        self.num_playoff_teams = schedule_settings['playoffTeamCount']
        self.playoff_seeding_tiebreaker = schedule_settings['playoffSeedingRule']
        
        # Not sure what this is at the moment. Usually is just 0
        self.playoffSeedingRuleBy = schedule_settings['playoffSeedingRuleBy']
        self.playoffSeedingRule = schedule_settings['playoffSeedingRule']
        self.num_reg_season_matchups = schedule_settings['matchupPeriodCount']
        
        self.reg_season_matchup_tiebreaker = scoring_settings['matchupTieRule']
        self.playoff_matchup_tiebreaker = scoring_settings['playoffMatchupTieRule']
        
        self.home_team_bonus = scoring_settings['homeTeamBonus']
        
        # Not sure what this is yet - returns "H2H_POINTS"
        self.scoring_type = scoring_settings['scoringType']
        
        
        self.firstScoringPeriod = status_settings['firstScoringPeriod']
        self.finalScoringPeriod = status_settings['finalScoringPeriod']
        
        # NEED THIS
        self.df_scoring_periods = self._pull_scoring_period_lookup(raw_settings_data, 
                                                                   self.num_reg_season_matchups)
    
    def _pull_divisions(self, raw_settings_data):
        """ Return dataframe containing the name and size of each division """
        
        divisions_list = raw_settings_data['settings']['scheduleSettings']['divisions']

        divisions = []
        for division in divisions_list:
            division_name = division['name']
            division_size = division['size']
            divisionId = division['id']

            divisions.append([division_name, division_size, divisionId])
            
        columns = ['division_name', 'division_size', 'divisionId']
        divisions = pd.DataFrame(divisions, columns=columns)

        return divisions
        
    def _pull_scoring_period_lookup(self, raw_settings_data, num_reg_season_matchups):
        """ Returns dataframe containing the scoring period mapped to matchup period """
        
        reg_season_matchups_pds = raw_settings_data['settings']['scheduleSettings']['matchupPeriods']
        
        scoring_pd_list = []
        for matchup_pd, scoring_pds in reg_season_matchups_pds.items():
            
            matchup_pd = int(matchup_pd)   
            for scoring_pd in scoring_pds:
                
                scoring_pd = int(scoring_pd)

                scoring_pd_list.append([scoring_pd, matchup_pd])
                
        columns = ['scoringPeriodId', 'macthupPeriodId']
        df = pd.DataFrame(scoring_pd_list, columns=columns)
        
        week_type_conditions = [
            (df['macthupPeriodId'] <= num_reg_season_matchups),
            (df['macthupPeriodId'] > num_reg_season_matchups)
        ]

        df['regular_season_ind'] = np.select(week_type_conditions, [1, 0])
                
        return df

check_settings = settingsData(2019, 28056918)
print(check_settings.df_divisions)
# check_settings.df_scoring_periods


  division_name  division_size  divisionId
0          East              3           0
1          West              3           1


In [169]:
class teamData():
    """ 
    Pulls relevant team data
    
    Notes:
        - The raw_mTeam_data['members'] data does NOT always include every team, but raw_mTeam_data['teams']
          does
        
    TODO:
        - Some repetitive code between the "_pull_team_members_nicknames" and "_pull_team_division" methods
    """
    
    def __init__(self, season_id, league_id):
        
        raw_mTeam_data = pull_data(season_id, league_id, params=(('view', 'mTeam')))
        
        self.season_id = season_id
        self.league_id = league_id
             
        self.df_members_names = self._pull_team_members_names(raw_mTeam_data)
        self.df_nicknames = self._pull_team_members_nicknames(raw_mTeam_data)
        self.df_divisions = self._pull_team_division(raw_mTeam_data)
        
        self.df_team_data = self._merge_all_data(raw_mTeam_data)
        
    def _pull_team_members_names(self, raw_mTeam_data):
        """ Creates a dataframe containing each team members full name """
        
        members_data = raw_mTeam_data['members']
        
        team_data = []
        for team in members_data:
            
            try:
                team_firstName = team['firstName']
                team_lastName = team['lastName']
                team_fullName = team_firstName + " " + team_lastName
            except:
                team_firstName = np.nan
                team_lastName = np.nan
                team_fullName = np.nan
            
            # id" in this portion of the API data returns the "primaryOwnerKey" from the "teams" section
            team_primaryOwnerKey = team['id']
            
            team_data.append([team_fullName, team_primaryOwnerKey])
            
        columns = ['team_fullName', 'team_primaryOwnerKey']
        df = pd.DataFrame(team_data, columns=columns)
        
        return df

    def _pull_team_members_nicknames(self, raw_mTeam_data):
        """ Creates a dataframe containing each team members nickname """
        
        members_data = raw_mTeam_data['teams']
        
        team_data = []
        for team in members_data:
            
            try:
                team_firstName = team['location']
                team_lastName = team['nickname']
                team_nickName = team_firstName + " " + team_lastName
            except:
                team_firstName = np.nan
                team_lastName = np.nan
                team_nickName = np.nan
                
            # Not sure why, but this isn't always included for a team
            try:
                team_primaryOwnerKey = team['primaryOwner']
            except:
                team_primaryOwnerKey = np.nan
            
            # id" in this portion of the API data returns the "primaryOwnerKey" from the "teams" section
            team_id = team['id']
            
            team_data.append([team_nickName, team_id, team_primaryOwnerKey])
            
        columns = ['team_nickName', 'team_id', 'team_primaryOwnerKey']
        df = pd.DataFrame(team_data, columns=columns)
        
        return df
    
    def _pull_team_division(self, raw_mTeam_data):
        """ Returns dataframe contaiing each teams division Id"""
        teams_data = raw_mTeam_data['teams']
        
        team_data = []
        for team in teams_data:
            
            team_divisionId = team['divisionId']
            team_id = team['id']
            
            # Not sure why, but this isn't always included for a team
            try:
                team_primaryOwnerKey = team['primaryOwner']
            except:
                team_primaryOwnerKey = np.nan
            
            team_data.append([team_divisionId, team_id, team_primaryOwnerKey])
            
        columns = ['team_divisionId', 'team_id', 'team_primaryOwnerKey']
        df = pd.DataFrame(team_data, columns=columns)
        
        return df
    
    def _merge_all_data(self, raw_mTeam_data):
        """ Returns dataframe with team member names and division Id merged together """
        
        df_team_members_names = self._pull_team_members_names(raw_mTeam_data)
        df_team_nicknames = self._pull_team_members_nicknames(raw_mTeam_data)
        df_team_divisions = self._pull_team_division(raw_mTeam_data)
        
        # 'team_primaryOwnerKey' is a redundant merge variable, but including to prevent two versions of it
        df = pd.merge(df_team_nicknames, df_team_divisions, 
                      on=['team_id', 'team_primaryOwnerKey'], how='outer')
        df = pd.merge(df, df_team_members_names, on='team_primaryOwnerKey', how='outer')
        
        return df


# check_team_data = teamData(2020, 48347143)
# check_team_data.df_team_data

In [168]:
# team_data = pull_data(2020, 48347143, params=(('view', 'mTeam')))

# print(team_data['members'][0]['id'])

# print(team_data['teams'][0].keys())

teamObj = teamData(2020, 24693394)

print(teamObj.df_nicknames)
print(teamObj.df_divisions)
print(teamObj.df_members_names)
teamObj.df_team_data

             team_nickName  team_id                    team_primaryOwnerKey
0               Fools Gold        1  {04D591E4-4B2D-4033-A33F-CE46730D5FFC}
1          BallzDeep Matta        2                                     NaN
2               Team smith        3  {58D7E40E-2EF7-468F-A202-3235E5BEB1B6}
3            Chicken Curry        4  {D0353F9F-4FD3-4DFE-B53F-9F4FD32DFE58}
4  CHAMP IS BACK GIMME $20        5                                     NaN
5     Chicago  Kareemville        6  {CBEEBF00-CFED-4BEF-BAB9-1CC54D7C6644}
6           Jersey Bombers        7  {DBF7700D-05E7-446C-902A-E99CED48D5BB}
7                Team Atiq        8  {E6379C8B-64BD-4AAE-B79C-8B64BD5AAE3F}
8      Fidel’s Bastard Son        9  {2702256E-1663-4AE0-8225-6E1663FAE01B}
   team_divisionId  team_id                    team_primaryOwnerKey
0                0        1  {04D591E4-4B2D-4033-A33F-CE46730D5FFC}
1                1        2                                     NaN
2                0        3  {58D7E4

,team_nickName,team_id,team_primaryOwnerKey,team_divisionId,team_fullName
0,Fools Gold,1,{04D591E4-4B2D-4033-A33F-CE46730D5FFC},0,benjamin marchese
1,BallzDeep Matta,2,NaN,1,NaN
2,CHAMP IS BACK GIMME $20,5,NaN,0,NaN
3,Team smith,3,{58D7E40E-2EF7-468F-A202-3235E5BEB1B6},0,john smith
4,Chicken Curry,4,{D0353F9F-4FD3-4DFE-B53F-9F4FD32DFE58},1,Arvin Rai
5,Chicago Kareemville,6,{CBEEBF00-CFED-4BEF-BAB9-1CC54D7C6644},1,Jason Lee
6,Jersey Bombers,7,{DBF7700D-05E7-446C-902A-E99CED48D5BB},0,patrick paragas
7,Team Atiq,8,{E6379C8B-64BD-4AAE-B79C-8B64BD5AAE3F},1,Anthony Atiq
8,Fidel’s Bastard Son,9,{2702256E-1663-4AE0-8225-6E1663FAE01B},0,Myles H


In [6]:
#####################################################################################
###### Pulls the Season/Team/Player/Week level data to get projected points #########
#####################################################################################

# file_dir = 'check.csv'
# check.to_csv(file_dir, mode='a', header=True, index=False)

def create_league_df(season_id, league_id, week_start=1, week_end=17):
    """ 
    Returns a dataframe containing the initial data necessary for the simulation
    for one league/year.
    
    Note: the settings_data and team_data objects are a bit overkill for this, but I'm using them
    in case there's any additional data that needs to be pulled in the future.
    
    Note: I'm creating files exclusively for Team and Settings data, so these variables will ultimatley be
    removed from this.
    """
    
    df_week_team_player = create_week_team_player_df(season_id, league_id, 
                                                     week_start=week_start, week_end=week_end)
    
    if len(df_week_team_player) == 0:
        return None
    
    else:
        settings_data = settingsData(season_id, league_id)
        team_data = teamData(season_id, league_id)

        df_divisions = settings_data.df_divisions[['divisionId', 'division_name']]
        
        df_team_data = team_data.df_team_data
        df_team_data = pd.merge(df_team_data, df_divisions,
                            left_on='team_divisionId', right_on='divisionId', how='left')

        df_scoring_periods = settings_data.df_scoring_periods
        df_week_team_player = pd.merge(df_week_team_player, df_scoring_periods,
                                      left_on='Week', right_on='scoringPeriodId', how='left')

        df_league_data = pd.merge(df_week_team_player, df_team_data,
                                left_on='Team', right_on='team_id', how='left')

        df_league_data.drop(['team_id', 'scoringPeriodId', 
                             'team_primaryOwnerKey', 'divisionId'], 1, inplace=True)
        df_league_data.rename(columns={'division_name': 'team_division_name'}, inplace=True)

    return df_league_data


def pull_all_leagues_projection_data(season_id, league_ids, index_start, index_end):
    """ Returns Season/Team/Player/Week level dataframe with each players actual and projected points """
    
    num_leagues = len(league_ids)
    
    if index_end > num_leagues:
        print("All Leagues have been processed")
        index_end = num_leagues    
    
    # This creates an empty dataframe that will be used for stacking
    get_cols_for_empty_df = create_league_df(2020, 48347143, week_end=1)
    
    columns = list(get_cols_for_empty_df.columns)
    columns.append('league_found_index')
    columns.append('league_id')
    
    df_league_data = pd.DataFrame(columns=columns)
    
    for i in range(index_start, index_end):
        league_id = league_ids[i]
        
        print(league_id, ": ", i)

        df_append_league_data = create_league_df(2020, league_id, week_end=17)
        
        if df_append_league_data is None:
            pass
        else:
            df_append_league_data['league_found_index'] = i

            df_append_league_data['league_id'] = league_id

            df_league_data = df_league_data.append(df_append_league_data)
    
    return df_league_data

check = create_league_df(2020, 48347143, week_end=1)
check

# league_ids = get_league_ids()
# all_leagues = pull_all_leagues_projection_data(2020, league_ids, 3000, 4000)

# all_leagues

,Week,Team,Player,Slot,Pos,Status,Proj,Actual,macthupPeriodId,regular_season_ind,team_fullName,team_divisionId,team_nickName,team_division_name
0,1,1,Alvin Kamara,2,RB,ACTIVE,17.406407,21.20,1,1,Connor DeLong,0,Team DeLong,East
1,1,1,Travis Kelce,6,TE,ACTIVE,14.886366,14.00,1,1,Connor DeLong,0,Team DeLong,East
2,1,1,Chris Carson,2,RB,ACTIVE,14.635647,21.60,1,1,Connor DeLong,0,Team DeLong,East
3,1,1,Calvin Ridley,4,WR,ACTIVE,11.783908,32.40,1,1,Connor DeLong,0,Team DeLong,East
4,1,1,Keenan Allen,23,Flex,ACTIVE,11.671297,5.70,1,1,Connor DeLong,0,Team DeLong,East
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
200,1,12,Austin Hooper,20,Bench,ACTIVE,6.595182,2.50,1,1,Matt Fleisher,0,Team Snacks,East
201,1,12,Nyheim Hines,20,Bench,ACTIVE,7.610436,23.30,1,1,Matt Fleisher,0,Team Snacks,East
202,1,12,Colts D/ST,16,Def,NA,8.107672,4.00,1,1,Matt Fleisher,0,Team Snacks,East
203,1,12,Derek Carr,20,Bench,ACTIVE,18.112839,13.56,1,1,Matt Fleisher,0,Team Snacks,East


In [141]:
#####################################################################################
################### Functions used to create the matchup data #######################
#####################################################################################

def create_matchup_df(matchup_data, playoff_week_start=14):
    """
    Returns a week/matchup level dataframe containing the total scores of each team
    Note that this requires the mMatchup filtered data
    """

    data = []

    for i, matchup_dict in enumerate(matchup_data['schedule']):
        matchup_period = matchup_dict['matchupPeriodId']
        week_number = matchup_dict['matchupPeriodId']
        playoffTierType = matchup_dict['playoffTierType']
        # NOTE: Need a way to properly assign week number. Below doesn't work because
        # 'pointsByScoringPeriod' is only available for weeks that have been played
        # print(week_number, ": ", matchup_dict['home']['pointsByScoringPeriod'].keys())

        # bye weeks cause the "away" key to be missing for the team with a bye
        try:
            teamId_away = int(matchup_dict['away']['teamId'])
            score_away = matchup_dict['away']['totalPoints']
        except KeyError:
            teamId_away = None
            score_away = None

        # applying this to home teams just in case
        try:
            teamId_home = int(matchup_dict['home']['teamId'])
            score_home = matchup_dict['home']['totalPoints']
        except KeyError:
            teamId_home = None
            score_home = None

        data.append([week_number, matchup_period, teamId_away, 
                     score_away, teamId_home, score_home, playoffTierType])

    df_matchup = pd.DataFrame(data, columns=['week_number', 'matchup_period', 'teamId_away', 'score_away',
                                             'teamId_home', 'score_home', 'playoffTierType']
                              )

    df_matchup['week_type'] = ['Regular' if w <= playoff_week_start else 'Playoffs' \
                               for w in df_matchup['week_number']]

    return df_matchup


def expand_matchup_data(matchup_data):
    """ Expands data to the week/team level """

    df_away_data = matchup_data.copy()
    df_home_data = matchup_data.copy()

    rename_a = {'teamId_away': 'teamId', 'score_away': 'score', 'teamId_home': 'teamId_opp', 'score_home': 'score_opp'}
    rename_h = {'teamId_home': 'teamId', 'score_home': 'score', 'teamId_away': 'teamId_opp', 'score_away': 'score_opp'}

    df_away_data.rename(columns=rename_a, inplace=True)
    df_away_data['home_or_away'] = 'away'

    df_home_data.rename(columns=rename_h, inplace=True)
    df_home_data['home_or_away'] = 'home'

    df_expanded = pd.concat([df_home_data, df_away_data])

    # update null values returned due to bye weeks
    df_expanded = df_expanded[df_expanded['teamId'].notnull()]
    df_expanded['teamId_opp'].fillna(-1, inplace=True)
    df_expanded['score_opp'].fillna(0, inplace=True)

    df_expanded['teamId'] = df_expanded['teamId'].astype('int')
    df_expanded['teamId_opp'] = df_expanded['teamId_opp'].astype('int')
    
    df_expanded.sort_values(['week_number', 'teamId'], inplace=True)

    return df_expanded


matchup_data = pull_data(2020, 48347143, params=[["view", "mMatchup"], ["view", "mMatchupScore"]])
# matchup_data = pull_data(2020, 48347143, params=[["view", "mMatchupScore"]])

df_matchup_data = create_matchup_df(matchup_data, playoff_week_start=14)
df_expanded_matchup = expand_matchup_data(df_matchup_data)

df_expanded_matchup

,week_number,matchup_period,teamId_opp,score_opp,teamId,score,playoffTierType,week_type,home_or_away
2,1,1,7,100.98,1,130.50,NONE,Regular,away
3,1,1,6,92.64,2,117.12,NONE,Regular,home
0,1,1,5,82.36,3,92.34,NONE,Regular,home
5,1,1,10,141.48,4,84.00,NONE,Regular,home
0,1,1,3,92.34,5,82.36,NONE,Regular,away
3,1,1,2,117.12,6,92.64,NONE,Regular,away
2,1,1,1,130.50,7,100.98,NONE,Regular,home
4,1,1,12,145.80,8,79.98,NONE,Regular,away
1,1,1,11,125.00,9,79.00,NONE,Regular,away
5,1,1,4,84.00,10,141.48,NONE,Regular,away


In [186]:
#####################################################################################
############### Pulls the team and settings data for all leagues ####################
#####################################################################################     

def pull_team_data(teamObj):
    """ Returns dataframe containing all relevant data for each team in a league/year """

    df = teamObj.df_team_data
    
    df['league_id'] = teamObj.league_id
    df['season_id'] = teamObj.season_id
    
    df = df[['season_id', 'league_id', 'team_id', 'team_primaryOwnerKey', 
             'team_fullName', 'team_nickName', 'team_divisionId']]
        
    return df

def pull_settings_data(settingsObj):
    """ Returns dataframe containing all relevant data for the settings of a league/year """

    st = settingsObj
    
    settings_list = [[st.playoffSeedingRule, st.playoffSeedingRuleBy, st.num_playoff_teams,
                    st.firstScoringPeriod, st.finalScoringPeriod, st.scoring_type, 
                    st.reg_season_matchup_tiebreaker, st.playoff_matchup_tiebreaker, st.home_team_bonus]]
    
    columns = ['settings_playoffSeedingRule', 'settings_playoffSeedigRuleBy', 'settings_num_playoff_teams',
               'settings_firstScoringPeriod', 'settings_finalScoringPeriod', 'settings_scoring_type', 
               'settings_reg_season_matchup_tiebreaker', 'settings_playoff_matchup_tiebreaker', 
               'settings_home_team_bonus']
    
    df = pd.DataFrame(settings_list, columns=columns)
    df['league_id'] = st.league_id
    df['season_id'] = st.season_id
    
    df = rearrange_df_columns(df, ['season_id', 'league_id'])
        
    return df


def pull_divisions_data(settingsObj):
    """ Returns dataframe containing all relvant division data for each league/year """
    
    df = settingsObj.df_divisions
    
    df['league_id'] = settingsObj.league_id
    df['season_id'] = settingsObj.season_id
    
    df = rearrange_df_columns(df, ['season_id', 'league_id'])
    
    return df


def pull_matchup_data(season_id, league_id, settingsObj):
    """ 
    Pulls the Week/Team level data for a league/year 
    Note: This data is being pulled primarily to be used to validate the Week/Team/Player projection data
    """
    
    params = [["view", "mMatchup"], ["view", "mMatchupScore"]]
    
    raw_matchup_data = pull_data(season_id, league_id, params=params)

    df_matchup_data = create_matchup_df(raw_matchup_data)
    df_expanded_matchup = expand_matchup_data(df_matchup_data)
    
    df_scoring_periods = settingsObj.df_scoring_periods

    # Merge the matchup period and regular season indicator onto the main dataframe
    df_expanded_matchup = pd.merge(df_expanded_matchup, df_scoring_periods,
                                  left_on='week_number', right_on='scoringPeriodId', how='left')
    
    df_expanded_matchup['league_id'] = league_id
    df_expanded_matchup['season_id'] = season_id
    
    keep_columns = ['season_id', 'league_id', 'week_number', 'macthupPeriodId', 'teamId', 'teamId_opp',
                   'score', 'score_opp', 'home_or_away', 'regular_season_ind', 'playoffTierType']
    
    df_expanded_matchup = df_expanded_matchup[keep_columns]
    
    return df_expanded_matchup


def pull_all_leagues_remaining_data(season_id, league_ids, index_start, index_end, league_id_example=48347143):
    """ Pulls the Team, Settings, Division, and Matchup data for every league found for one season """
    
    num_leagues = len(league_ids)
    
    if index_end > num_leagues:
        print("All Leagues have been processed")
        index_end = num_leagues   
    
    # Create the initial datasets using a league that is known to be populated
    teamObj = teamData(season_id, league_id_example)
    settingsObj= settingsData(season_id, league_id_example)

    df_team_stack = pull_team_data(teamObj)
    df_settings_stack = pull_settings_data(settingsObj)
    df_divisions_stack = pull_divisions_data(settingsObj)
    df_matchup_data_stack = pull_matchup_data(season_id, league_id_example, settingsObj)
    
    df_stack_dict = {'df_team_stack': df_team_stack, 'df_settings_stack': df_settings_stack, 
               'df_divisions_stack': df_divisions_stack, 'df_matchup_data_stack': df_matchup_data_stack}
    
    for df_name, df in df_stack_dict.items():
        # This creates an empty dataframe that will be used for stacking
        columns = list(df.columns)
        df_stack_dict[df_name] = pd.DataFrame(columns=columns)
    
    for i in range(index_start, index_end):
        
        league_id = league_ids[i]
        
        print(league_id, ": ", i)
        
        teamObj = teamData(season_id, league_id)
        settingsObj= settingsData(season_id, league_id)

        df_team = pull_team_data(teamObj)
        df_settings = pull_settings_data(settingsObj)
        df_divisions = pull_divisions_data(settingsObj)
        df_matchup_data = pull_matchup_data(season_id, league_id, settingsObj)

        df_dict = {'df_team': df_team, 'df_settings': df_settings, 
                   'df_divisions': df_divisions, 'df_matchup_data': df_matchup_data}

        for df_name, df in df_dict.items():
            if df is None:
                pass
            else:
                df_stack_name = df_name + "_stack"
                df_stack_dict[df_stack_name] = df_stack_dict[df_stack_name].append(df)
                

    return df_stack_dict

############################### THIS CHECKS THE FUNCTIONS ##########################
# pd.options.display.max_rows = None
# league = 24693394

# teamObj = teamData(2020, league)
# settingsObj= settingsData(2020, league)

# df_team = pull_team_data(teamObj)
# df_settings = pull_settings_data(settingsObj)
# df_matchup = pull_matchup_data(2020, league, settingsObj)

# df_team

############################ THIS CREATES ALL THE DATAFRAMES ########################
file = "/home/cdelong/Python-Projects/FF-Web-App/Repo-Work/delt-ff-dashboard/simulation_work/Leagues_Found_Cleaned.csv"

df_leagues_found = pd.read_csv(file)
df_leagues_found = df_leagues_found.loc[df_leagues_found['seasonId'] == 2020]

leagues_found_list = list(df_leagues_found['leagueId'])

df_dict = pull_all_leagues_remaining_data(2020, leagues_found_list, 0, 1)

df_dict['df_matchup_data_stack']


All Leagues have been processed
61596907 :  2000
61597182 :  2001
61597794 :  2002
61598037 :  2003
61598069 :  2004
61598077 :  2005
61598674 :  2006
61598984 :  2007
61599932 :  2008
61600090 :  2009
61600417 :  2010
61600497 :  2011
61601165 :  2012
61601391 :  2013
61601644 :  2014
61601734 :  2015
61601973 :  2016
61602546 :  2017
61603014 :  2018
61603174 :  2019
61603606 :  2020
61603938 :  2021
61604647 :  2022
61605680 :  2023
61606377 :  2024
61606672 :  2025
61607135 :  2026
61607620 :  2027
61607735 :  2028
61607787 :  2029
61608115 :  2030
61608552 :  2031
61608625 :  2032
61608634 :  2033
61608998 :  2034
61609409 :  2035
61609668 :  2036
61609700 :  2037
61610183 :  2038
61610452 :  2039
61610724 :  2040
61611279 :  2041
61611344 :  2042
61611655 :  2043
61611988 :  2044
61612931 :  2045
61613524 :  2046
61613704 :  2047
61614049 :  2048
61614120 :  2049
61614173 :  2050
61614374 :  2051
61614702 :  2052
61615056 :  2053
61615062 :  2054
61615148 :  2055
61615987 :  2056

,season_id,league_id,week_number,macthupPeriodId,teamId,teamId_opp,score,score_opp,home_or_away,regular_season_ind,playoffTierType
0,2020,61596907,1,1,1,3,122.50,135.08,away,1,NONE
1,2020,61596907,1,1,2,4,115.58,98.90,away,1,NONE
2,2020,61596907,1,1,3,1,135.08,122.50,home,1,NONE
3,2020,61596907,1,1,4,2,98.90,115.58,home,1,NONE
4,2020,61596907,1,1,5,7,167.04,168.38,away,1,NONE
...,...,...,...,...,...,...,...,...,...,...,...
160,2020,68999315,15,14,7,11,144.04,217.28,home,0,LOSERS_CONSOLATION_LADDER
161,2020,68999315,15,14,8,5,150.76,255.84,home,0,WINNERS_CONSOLATION_LADDER
162,2020,68999315,15,14,9,-1,242.98,0.00,home,0,LOSERS_CONSOLATION_LADDER
163,2020,68999315,15,14,10,3,194.12,165.42,home,0,LOSERS_CONSOLATION_LADDER


In [187]:
#####################################################################################
########## Creates the csvs of all relevant data for each league found ##############
#####################################################################################

def write_df_to_csv(df_dict, file_names, directory):
    """ Writes a set of dataframes to csv files """
    
    if directory[-1] != "/":
        directory = directory + "/"        
    
    list_index = 0
    for df in df_dict.values():
        file_name = file_names[list_index]
        
        if file_name[-4:].lower() != ".csv":
            file_name = file_name + ".csv"
            
        full_path = directory + file_name
        
        df.to_csv(full_path, index=False
                 , mode='a', header=False
                 )
        
        list_index += 1
        

# file_names = ['TeamData_Season_League_Team', 'SettingsData_Season_League', 
#               'DivisionData_Season_League_Division', 'MatchupData_Season_League_Team_Week']
        
# write_df_to_csv(df_dict, file_names, "/home/cdelong/Python-Projects/FF-Web-App/Simulation-Data/")

In [344]:
all_leagues_processed = pd.read_csv('All_Leagues_Data.csv')

# leagues_found = pd.read_csv('Leagues_Found.csv')

leagues_processed_list = all_leagues_processed.groupby(['league_id'], as_index=False).size()

print(len(leagues_processed_list))
print(len(all_leagues))
print(len(all_leagues_processed))

2170
1248359
5603054


/******************************************************************************************************/
/******************************************************************************************************/
/***************************************** SCRATCH WORK ***********************************************/
/******************************************************************************************************/
/******************************************************************************************************/

In [55]:

file = "/home/cdelong/Python-Projects/FF-Web-App/Simulation-Data/ProjPointsData_Season_Team_Player_Week.csv"

df_leagues_found = pd.read_csv(file)

original = 'Leagues_Found.csv'

df_original_leagues_found = pd.read_csv(original)

df_leagues_found

# df = pd.merge(df_team_members_names, df_team_divisions, on='team_primaryOwnerKey')

# df_leagues_found = df_leagues_found.loc[df_leagues_found['seasonId'] == 2020]


,Week,Team,Player,Slot,Pos,Status,Proj,Actual,macthupPeriodId,regular_season_ind,team_fullName,team_divisionId,team_division_name,league_found_index,league_id
0,1,1,Alvin Kamara,2,RB,OUT,17.406407,21.2,1.0,1.0,Connor DeLong,0.0,East,0,48347143
1,1,1,Travis Kelce,6,TE,ACTIVE,14.886366,14.0,1.0,1.0,Connor DeLong,0.0,East,0,48347143
2,1,1,Chris Carson,2,RB,ACTIVE,14.635647,21.6,1.0,1.0,Connor DeLong,0.0,East,0,48347143
3,1,1,Calvin Ridley,4,WR,ACTIVE,11.783908,32.4,1.0,1.0,Connor DeLong,0.0,East,0,48347143
4,1,1,Keenan Allen,23,Flex,OUT,11.671297,5.7,1.0,1.0,Connor DeLong,0.0,East,0,48347143
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5603049,17,10,Chase Claypool,4,WR,ACTIVE,10.377665,21.1,15.0,0.0,Quinn McIlvanie,1.0,West,3616,61647429
5603050,17,10,Noah Fant,20,Bench,ACTIVE,11.205609,8.8,15.0,0.0,Quinn McIlvanie,1.0,West,3616,61647429
5603051,17,10,Giovani Bernard,20,Bench,ACTIVE,11.558791,0.7,15.0,0.0,Quinn McIlvanie,1.0,West,3616,61647429
5603052,17,10,Kalen Ballage,20,Bench,ACTIVE,7.053212,9.6,15.0,0.0,Quinn McIlvanie,1.0,West,3616,61647429


In [43]:
#####################################################################################
####### Create a new Leagues Found file that excludes leagues without data ##########
#####################################################################################

all_league_ids = df_leagues_found.groupby(['league_id'], as_index=False).size()

new_leagues_found = pd.merge(all_league_ids['league_id'], 
                             df_original_leagues_found, 
                             left_on='league_id', right_on='leagueId',
                             how='left')

new_leagues_found = new_leagues_found[['leagueId', 'number_of_teams', 'startId', 'endId', 
                                      'seasonId', 'status_code']]

new_leagues_found.to_csv('Leagues_Found_Cleaned.csv')

In [11]:
# 28056918
# 48347143
# settings_data = pull_settings_data(2020, league_id=28056918)[0]

# Need to loop through scoring periods, but can do all teams with one pull

request_params = (("view", "mMatchup"), ("view", "mMatchupScore"))
roster_data = pull_data(2020, 48347143, params=request_params)

print(roster_data.keys(), "\n")

# print(roster_data['gameId'], "\n")

print(roster_data['schedule'][80].keys(), "\n")

print(roster_data['schedule'][80]['playoffTierType'], "\n")

# print(roster_data['teams'][0].keys(), "\n")

# print(roster_data['teams'][0]['roster']['entries'][0].keys(), "\n")

# print(roster_data['teams'][0]['roster']['entries'][0]['lineupSlotId'])

# playerPool = roster_data['teams'][0]['roster']['entries'][0]['playerPoolEntry']

# print("CHECK THIS: ", playerPool.keys(), "\n")

# print(playerPool['player'].keys(), "\n")

# print(playerPool['player']['rankings'].keys(), "\n")

# print(playerPool['player']['rankings']['0'], "\n")


dict_keys(['draftDetail', 'gameId', 'id', 'schedule', 'scoringPeriodId', 'seasonId', 'segmentId', 'status', 'teams']) 

dict_keys(['away', 'home', 'id', 'matchupPeriodId', 'playoffTierType', 'winner']) 

WINNERS_BRACKET 



In [129]:
def pull_week_team_player_data_new(seasonId, leagueId):

    data = []
    for week_number in range(1, 5):
        
        request_params = (("forTeamId", 1), ("view", "mRoster"), ("scoringPeriodId", week_number),
                         ("view", "overview"), ("statSplit", "singleScoringPeriod"))

        roster_data = pull_data(seasonId, league_id=leagueId, dict_params=request_params)[0]
        
        for player in roster_data['teams'][0]['roster']['entries']:
            
            slot = player['lineupSlotId']
            position  = slotcodes[slot]
            
            playerPool = player['playerPoolEntry']
            
            week_number_str = str(week_number)
            
            try:
                alt_slot = playerPool['player']['rankings'][week_number_str][0]['slotId']
                alt_position = slotcodes[alt_slot]
            except:
                alt_position = "N/A"
            
            player_name = player['playerPoolEntry']['player']['fullName']
            
            data.append([player_name, position, alt_position, week_number])
            
    columns = ['player_name', 'position', 'alt_position', 'week_number']
    
    df = pd.DataFrame(data, columns=columns)
    
    return df
            
check = pull_week_team_player_data_new(2019, 48347143)

check
    
# print(len(roster_data['teams'][0]['roster']['entries']))

,player_name,position,alt_position,week_number
0,Christian McCaffrey,RB,RB,1
1,Leonard Fournette,RB,RB,1
2,Mike Evans,IR,WR,1
3,Tyler Lockett,WR,WR,1
4,Derrius Guice,IR,RB,1
5,Damien Williams,Bench,RB,1
6,Ronald Jones II,Bench,RB,1
7,Jameis Winston,QB,QB,1
8,49ers D/ST,Bench,Def,1
9,Kenyan Drake,Flex,RB,1


In [42]:

@sleep_and_retry
@limits(calls=6000, period=600)
def pull_data_old(year, league_id = 0, dict_params = {}):
    """ Returns a JSON object containing the data pulled APIs url """
    if league_id == 0:
        url = "https://fantasy.espn.com/apis/v3/games/ffl/seasons/" + str(year)
    else:
        if year < 2020:
            url = "https://fantasy.espn.com/apis/v3/games/ffl/leagueHistory/" + \
                  str(league_id) + "?seasonId=" + str(year)
        else:
            url = "https://fantasy.espn.com/apis/v3/games/ffl/seasons/" + \
                  str(year) + "/segments/0/leagues/" + str(league_id)     
        
    r = requests.get(url, params=dict_params)
    
    status_code = r.status_code
    
    if r.status_code == 200:
        pass
    else:
        if r.status_code == 429:
            print("429 error")

        return None   
    
    # 2020 url returns JSON object while prior years return it in a list 
    if year < 2020:
        d = r.json()[0]
    else:
        d = r.json()
        
    r.close()
        
    return [d, status_code

200
404


In [68]:
file = '/home/cdelong/python_projects/ff_web_app/delt_ff_standings/Leagues_Found.csv'

leagues_found = pd.read_csv(file)

leagueIds = list(leagues_found['leagueId'])

status_codes = []
for league in leagueIds[0:100]:
    status_code = pull_data(2019, league_id=league)[1]
    
    status_codes.append([league, status_code])

d_status_codes = pd.DataFrame(status_codes, columns=['leagueId', 'status_codes'])

d_status_codes

,leagueId,status_codes
0,48347143,200
1,48347191,200
2,48347259,200
3,48347360,200
4,48347430,200
...,...,...
95,28056904,200
96,28056918,200
97,28056993,200
98,28057018,200
